# File `.p` Conversion from COMIC `.h5`

In [ ]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from envs.humanoid import HumanoidTracking
import numpy as np
import mediapy as media
import jax.numpy as jp
import mujoco
from dm_control import mjcf

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from dm_control import composer
from dm_control.locomotion import arenas
from dm_control.locomotion import walkers
from walker import Rat
from cmu_humanoid import CMUHumanoidPositionControlled
from dm_control.locomotion.mocap import props
from dm_control.locomotion.tasks.reference_pose import tracking
from dm_control.locomotion.tasks.reference_pose import types
from dm_control.utils import io as resources
import os
import h5py
import pickle

In [ ]:
# Prints the h5 file hierarchy and the dataset shapes
def print_hierarchy(h5file, indent=""):
    for key, item in h5file.items():
        if isinstance(item, h5py.Group):
            print(f"{indent}{key} (Group)")
            print_hierarchy(item, indent + "  ")
        elif isinstance(item, h5py.Dataset):
            print(f"{indent}{key} (Dataset): {item.shape}")
            
filename = "clips/test_trajectories.h5"
with h5py.File(filename, "r") as f:
    print(f"Contents of {filename}:")
    print_hierarchy(f)

In [ ]:
def extract_h5_to_dict(file):
    """Recursively extract all datasets from the HDF5 group into a dictionary."""
    data_dict = {}
    for key in file.keys():
        if isinstance(file[key], h5py.Dataset):
            # Store data as a numpy array
            data_dict[key] = file[key][:]
        elif isinstance(file[key], h5py.Group):
            # Recursively process each group
            data_dict.update({f"{key}/{subkey}": subvalue for subkey, subvalue in extract_h5_to_dict(file[key]).items()})
    return data_dict

# Path to your .h5 file
file_path = 'clips/test_traj.h5'

# Open the .h5 file and extract all data into a dictionary
with h5py.File(file_path, 'r') as file:
    all_data_dict = extract_h5_to_dict(file)

In [ ]:
all_data_dict.keys()

In [ ]:
# Dictionary to store specific data following the provided structure
structured_data = {
    'angular_velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/angular_velocity', None).T,
    'appendages': all_data_dict.get('cmuv2019_001/walkers/walker_0/appendages', None).T,
    'body_positions': all_data_dict.get('cmuv2019_001/walkers/walker_0/body_positions', None).T,
    'body_quaternions': all_data_dict.get('cmuv2019_001/walkers/walker_0/body_quaternions', None).T,
    'center_of_mass': all_data_dict.get('cmuv2019_001/walkers/walker_0/center_of_mass', None).T,
    'end_effectors': all_data_dict.get('cmuv2019_001/walkers/walker_0/end_effectors', None).T,
    'joints': all_data_dict.get('cmuv2019_001/walkers/walker_0/joints', None).T,
    'joints_velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/joints_velocity', None).T,
    'markers': all_data_dict.get('cmuv2019_001/walkers/walker_0/markers', None),
    'position': all_data_dict.get('cmuv2019_001/walkers/walker_0/position', None).T,
    'quaternion': all_data_dict.get('cmuv2019_001/walkers/walker_0/quaternion', None).T,
    'scaling': all_data_dict.get('cmuv2019_001/walkers/walker_0/scaling', None),
    'velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/velocity', None).T
}

# Optionally, print the data for verification
for key, value in structured_data.items():
    print(f"{key}: {type(value)}")

In [ ]:
[np.array(structured_data[item]).shape for item in structured_data]

In [ ]:
[item for item in structured_data]

In [ ]:
import mocap_preprocess as mp
mp.save_dataclass_pickle('clips/test_traj.p', structured_data)

# Checking if shape mataches
1. for qpos stacks, `angular_velocity`, `veclocity` matches, `joints_velocity` doesn't

In [ ]:
file_path = 'clips/humanoid_traj.p'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

print(type(data))
print(data.joints_velocity.shape)

In [ ]:
file_path = 'clips/test_traj.p'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

print(type(data))
print(data.joints_velocity.shape)

# Our `humanoid_traj.p` to `.h5` For COMIC

In [ ]:
cat = ['angular_velocity',
 'appendages',
 'body_positions',
 'body_quaternions',
 'center_of_mass',
 'end_effectors',
 'joints',
 'joints_velocity',
 'markers',
 'position',
 'quaternion',
 'scaling',
 'velocity']

# Path to the pickle file and the desired output HDF5 file path
pickle_file_path = 'clips/humanoid_traj.p'
hdf5_file_path = 'humanoid_traj.h5'

# Load the data from the pickle file
with open(pickle_file_path, 'rb') as file:
    data = pickle.load(file)

# Create an HDF5 file and store the data
with h5py.File(hdf5_file_path, 'w') as hdf_file:
    for mocap in cat:
        # Assuming each value is an array-like object that can be directly stored in HDF5
        if isinstance(value, np.ndarray):
            hdf_file.create_dataset(cat, data=data.mocap)
        else:
            print(f"Skipping key {key} as it is not an array.")
print(f"Data from {pickle_file_path} has been successfully written to {hdf5_file_path}.")